# Handling Gaze Events

## What you will learn in this tutorial:

* how to detect different events using different algorithms like IDT, IVT and microsaccades
* how to compute event properties like peak velocity and amplitude
* how to save and load your event data


## Detecting Gaze Events
## Preparations
At first we import `pymovements` as the alias `pm` for convenience.

In [ ]:
import pymovements as pm

Then we download a dataset `ToyDataset` and load its data:

In [ ]:
dataset = pm.Dataset('ToyDataset', path='data/ToyDataset')
dataset.download()
dataset.load()

The dataset consist of gaze data in 20 files (check `Dataset/gaze` above). Every `Gaze` has some samples with six columns (check `Gaze/samples`): [time, stimuli_x, stimuli_y, text_id, page_id, pixel]. The `Gaze/events` DataFrame is empty so far. To be able to calculate events, we need to do some basic preprocessing, which will add new columns to the dataset samples DataFrame:

* `pix2deg()`: adds `position` column with degrees from the screen center needed by the `idt` algorithm
* `pos2vel()`: adds `velocity` column with gaze velocities needed by `microsaccades` and `ivt` algorithms


In [ ]:
dataset.pix2deg()
dataset.pos2vel('smooth')
dataset.gaze[0]

Now every `Gaze/samples` DataFrame has two more columns: position and velocity which will be used by the event detection algorithms.

## Detecting Events

*pymovements* provides a range of event detection methods for several types of gaze events.

See the reference for [pymovements.events](https://pymovements.readthedocs.io/en/latest/reference/pymovements.events.html) to get an overview of all the supported methods.

For this tutorial we will use the I-DT and I-VT (`idt` and `ivt`) algorithms for detecting fixations and the `microsaccades` algorithm for detecting saccades.

Let's start with fixations detection using the `idt` algorithm with the `dispersion_threshold` equal to 2.7:

In [ ]:
dataset.detect_events('idt', dispersion_threshold=2.7)

The detected events are added as rows with the name `fixation` to the event dataframe:

In [ ]:
dataset.events[0]

As you can see, 56 fixations were found for the first file.

Now let's try another algorithm `ivt` with velocity_threshold=20. Because we don't want to mix fixations found by different algorithms we add `name` parameter with 'fixation.ivt'

In [ ]:
dataset.detect_events('ivt', velocity_threshold=20, name='fixation.ivt')
dataset.events[0]

Now we have additional rows with name='fixations.ivt'.

Let's try to use the `microsaccades` algorithm to detect fixations.

In [ ]:
dataset.detect_events('microsaccades', minimum_duration=12)

The detected events are added as rows with the name `saccade` to the event dataframe:

In [ ]:
dataset.events[0]

Now there are three sets of events in the `dataset.events` DataFrame with different values in the 'name' column:

In [ ]:
set(dataset.events[0].frame['name'])

## Computing Event Properties

*pymovements* provides a range of event properties.

See the reference for [pymovements.events](https://pymovements.readthedocs.io/en/latest/reference/pymovements.events.html) to get an overview of all the supported properties.

For this tutorial we will compute several properties of saccades.

We start out with the peak velocity:

In [ ]:
dataset.compute_event_properties("peak_velocity")

dataset.events[0]

Check above that a new column with the name `peak_velocity` has appeared in the event DataFrame.

We can also pass a list of properties. Let's add the amplitude and dispersion:

In [ ]:
dataset.compute_event_properties(["amplitude", "dispersion"])

dataset.events[0]

This way we can compute all of our desired properties in a single run.

## Saving Event Data

Saving your event data is as simple as:

In [ ]:
dataset.save_events()

All of the event data is saved into this directory:

In [ ]:
dataset.paths.events

Let's confirm it by printing all files in this directory:

In [ ]:
print(list(dataset.paths.events.glob('*/*/*')))

All files have been saved into the `Dataset.paths.events` as files in [Feather format](https://arrow.apache.org/docs/python/feather.html). 

If we want to save the data into an alternative directory and also use a different file format like `csv` we can use the following:

In [ ]:
dataset.save_events(events_dirname='events_csv', extension='csv')

Let's confirm again by printing all the new files in this alternative directory:

In [ ]:
alternative_dirpath = dataset.path / 'events_csv'
print(list(alternative_dirpath.glob('*/*/*')))

### Loading Previously Computed Events Data

Let's initialize a new dataset object from the same `ToyDataset`.

In [ ]:
preprocessed_dataset = pm.Dataset('ToyDataset', path='data/ToyDataset')

When we load the dataset using `load()` without any parameters there will be no events loaded:

In [ ]:
preprocessed_dataset.load()

But when we load it with the `events=True` parameter the events will be loaded:


In [ ]:
preprocessed_dataset.load(events=True)

By default, the `events` directory and the `feather` extension will be chosen.

In case of alternative directory names or other file formats you can use the following:

In [ ]:
preprocessed_dataset.load(
    events=True,
    events_dirname='events_csv',
    extension='csv',
)
dataset.events[0]

## What you have learned in this tutorial:

* detecting different events with different algorithms by using `Dataset.detect_events()`
* computing event properties by using `Dataset.compute_event_properties()`
* saving your preprocesed data using `Dataset.save_preprocessed()`
* load your preprocesed data using `Dataset.load(events=True)`
* using custom directory names by specifying `preprocessed_dirname`
* using other file formats than the default `feather` format by specifying `extension`